In [1]:
import pandas as pd
import numpy as np

In [17]:
data = {'Name': ['A', 'B', 'C'], 'Start_time': ['2017-12-15', '2017-12-15', '2017-12-15'], 'End_time': ['2018-01-17', '2017-12-22', '2018-05-22']}
df = pd.DataFrame(data)

In [3]:
# df = df.astype({'Start_time': 'datetime64[ns]', 'End_time': 'datetime64[ns]'})

In [4]:
df.dtypes

Name                  object
Start_time    datetime64[ns]
End_time      datetime64[ns]
dtype: object

In [5]:
df

,Name,Start_time,End_time
0,A,2017-12-15,2018-01-17
1,B,2017-12-15,2017-12-22
2,C,2017-12-15,2018-05-22


In [6]:
from pandas.tseries.offsets import *
df['LastDay'] = df['Start_time'] + MonthEnd()

In [7]:
df

,Name,Start_time,End_time,LastDay
0,A,2017-12-15,2018-01-17,2017-12-31
1,B,2017-12-15,2017-12-22,2017-12-31
2,C,2017-12-15,2018-05-22,2017-12-31


In [8]:
df.End_time <= df.LastDay 

0    False
1     True
2    False
dtype: bool

In [9]:
dList = []
vList = []
d = {}

def buildDict(row):   
    for x in pd.date_range(row["Start_time"],row["End_time"], freq='MS'):  #build a range for each row
        dList.append(x)  #date list
        vList.append(row["Name"]) #value list 

df.apply(buildDict,axis=1) #each row in df is passed to buildDict

#this d will be used to create our new frame
d["date"] = dList
d["value"] = vList

In [10]:
pd.DataFrame(d)

,date,value
0,2018-01-01,A
1,2018-01-01,C
2,2018-02-01,C
3,2018-03-01,C
4,2018-04-01,C
5,2018-05-01,C


In [11]:
# import datetime

# # borrowed from https://stackoverflow.com/a/13565185
# # as noted there, the calendar module has a function of its own
# def last_day_of_month(any_day):
#     next_month = any_day.replace(day=28) + datetime.timedelta(days=4)  # this will never fail
#     return next_month - datetime.timedelta(days=next_month.day)

# begin = "2018-02-15"
# end = "2018-04-23"

# def monthlist(begin,end):
#     begin = datetime.datetime.strptime(begin, "%Y-%m-%d")
#     end = datetime.datetime.strptime(end, "%Y-%m-%d")

#     result = []
#     while True:
#         if begin.month == 12:
#             next_month = begin.replace(year=begin.year+1,month=1, day=1)
#         else:
#             next_month = begin.replace(month=begin.month+1, day=1)
#         if next_month > end:
#             break
#         result.append ([begin.strftime("%Y-%m-%d"),last_day_of_month(begin).strftime("%Y-%m-%d")])
#         begin = next_month
#     result.append ([begin.strftime("%Y-%m-%d"),end.strftime("%Y-%m-%d")])
#     return result

In [20]:
import datetime

# borrowed from https://stackoverflow.com/a/13565185
# as noted there, the calendar module has a function of its own
def last_day_of_month(any_day):
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)  # this will never fail
    return next_month - datetime.timedelta(days=next_month.day)

def monthlist(row):
    begin = datetime.datetime.strptime(row.Start_time, "%Y-%m-%d")
    end = datetime.datetime.strptime(row.End_time, "%Y-%m-%d")

    result = []
    while True:
        if begin.month == 12:
            next_month = begin.replace(year=begin.year+1,month=1, day=1)
        else:
            next_month = begin.replace(month=begin.month+1, day=1)
        if next_month > end:
            break
        result.append ([begin.strftime("%Y-%m-%d"),last_day_of_month(begin).strftime("%Y-%m-%d")])
        begin = next_month
    result.append ([begin.strftime("%Y-%m-%d"),end.strftime("%Y-%m-%d")])
    return result

In [12]:
date_list = monthlist(begin,end)
print (date_list)

[['2018-02-15', '2018-02-28'], ['2018-03-01', '2018-03-31'], ['2018-04-01', '2018-04-23']]


In [35]:
split = pd.DataFrame(df.apply(monthlist, axis=1).tolist())

In [37]:
split

,0,1,2,3,4,5
0,"[2017-12-15, 2017-12-31]","[2018-01-01, 2018-01-17]",None,None,None,None
1,"[2017-12-15, 2017-12-22]",None,None,None,None,None
2,"[2017-12-15, 2017-12-31]","[2018-01-01, 2018-01-31]","[2018-02-01, 2018-02-28]","[2018-03-01, 2018-03-31]","[2018-04-01, 2018-04-30]","[2018-05-01, 2018-05-22]"


In [43]:
test = pd.concat([df, split], axis=1)
test

,Name,Start_time,End_time,0,1,2,3,4,5
0,A,2017-12-15,2018-01-17,"[2017-12-15, 2017-12-31]","[2018-01-01, 2018-01-17]",None,None,None,None
1,B,2017-12-15,2017-12-22,"[2017-12-15, 2017-12-22]",None,None,None,None,None
2,C,2017-12-15,2018-05-22,"[2017-12-15, 2017-12-31]","[2018-01-01, 2018-01-31]","[2018-02-01, 2018-02-28]","[2018-03-01, 2018-03-31]","[2018-04-01, 2018-04-30]","[2018-05-01, 2018-05-22]"


In [49]:
pd.melt(test, id_vars=['Name']).dropna().sort_values(by='Name')

,Name,variable,value
0,A,Start_time,2017-12-15
3,A,End_time,2018-01-17
6,A,0,"[2017-12-15, 2017-12-31]"
9,A,1,"[2018-01-01, 2018-01-17]"
1,B,Start_time,2017-12-15
4,B,End_time,2017-12-22
7,B,0,"[2017-12-15, 2017-12-22]"
2,C,Start_time,2017-12-15
5,C,End_time,2018-05-22
8,C,0,"[2017-12-15, 2017-12-31]"
